In [4]:
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch

# Load the dataset label: review text, ":" is a separator
data = []
with open('C:/Users/dzyyu/Arbeit/MasterArbeit/MASTERARBEIT/Datasets/classification/Review/reviewshort.csv', 'r') as file:
    for line in file:
        if ':' in line:
            label, text = line.strip().split(':', 1)
            data.append({'label': label, 'text': text})

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Tokenization and encoding for text with BERT. TPOT does not provide text encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['last_hidden_state'][:,0,:].numpy()

# Apply BERT encoding to the text
df['embedding'] = df['text'].apply(get_bert_embedding)

# Flatten the embeddings
X = np.vstack(df['embedding'].to_numpy())

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a TPOT classifier without custom configuration
tpot = TPOTClassifier(generations=5, population_size=50, verbosity=2, max_time_mins=15)

# Fit the classifier on the training data
tpot.fit(X_train, y_train)

# Evaluate the model
print("Accuracy", tpot.score(X_test, y_test))

# Export the best pipeline
tpot.export('tpot_2008_pipeline.py')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Version 0.12.0 of tpot is outdated. Version 0.12.1 was released Tuesday August 15, 2023.


Optimization Progress:   0%|          | 0/50 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.5875

Generation 2 - Current best internal CV score: 0.5875

Generation 3 - Current best internal CV score: 0.5875

Generation 4 - Current best internal CV score: 0.5875

Generation 5 - Current best internal CV score: 0.6125

Best pipeline: SGDClassifier(input_matrix, alpha=0.01, eta0=0.1, fit_intercept=True, l1_ratio=1.0, learning_rate=constant, loss=modified_huber, penalty=elasticnet, power_t=0.1)
Accuracy 0.4


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
import numpy as np
from transformers import BertTokenizer, BertModel
from sklearn.model_selection import train_test_split
from tpot import TPOTClassifier
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
import torch
from sklearn.metrics import accuracy_score

# Prediction done with the model choosen by TPOT. Text encoding done manually, since TPOT doe not support text encoding.
data = []
with open('C:/Users/dzyyu/Arbeit/MasterArbeit/MASTERARBEIT/Datasets/classification/Review/reviewshort.csv', 'r') as file:
    for line in file:
        if ':' in line:
            label, text = line.strip().split(':', 1)
            data.append({'label': label, 'text': text})

# Convert to a pandas DataFrame
df = pd.DataFrame(data)

# Tokenization and encoding
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

#encodig with BERT
def get_bert_embedding(text):
    inputs = tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs['last_hidden_state'][:,0,:].numpy()

# Apply BERT encoding to the text
df['embedding'] = df['text'].apply(get_bert_embedding)

# Flatten the embeddings
X = np.vstack(df['embedding'].to_numpy())

# Convert labels into numeric format
#le = LabelEncoder()
#y = le.fit_transform(df['label'])
#tpot_data=df

# Train-test split
training_features, testing_features, training_target, testing_target = \
            train_test_split(X, y, random_state=None)

# use the algorithm choosen by TPOT   
exported_pipeline = SGDClassifier(input_matrix, alpha=0.01, eta0=0.1, fit_intercept=True, l1_ratio=1.0, learning_rate=constant, loss=modified_huber, penalty=elasticnet, power_t=0.1)
exported_pipeline.fit(training_features, training_target)
results = exported_pipeline.predict(testing_features)
print(testing_target)
print(results)

#decode results
#decoded_results = le.inverse_transform(results)
#print(decoded_results)
accuracy = accuracy_score(testing_target, results)
print(f"Accuracy Score: {accuracy * 100:.2f}%")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[3 4 3 3 4 1 3 4 3 2 4 1 3 1 2 4 4 3 4 4 4 4 4 1 1]
[4 4 4 3 3 3 4 3 4 4 3 3 4 4 3 4 4 4 4 4 4 4 4 4 2]
Accuracy Score: 36.00%
